In [22]:
# Data processing
import pandas as pd
# Scientific computing
import scipy
# Regular expression operations
import re
# Common string operations
import string 

# Natural language processing
import nltk 
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

nltk.download('wordnet') 
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package wordnet to C:\Users\BEA
[nltk_data]     YUKDAWAN\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\BEA
[nltk_data]     YUKDAWAN\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\BEA
[nltk_data]     YUKDAWAN\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [23]:
# read the data
data = pd.read_csv('../data/data.csv')
data.head()

,subreddit,id,timestamp,permalink,author,tag,title,body,comments,score
0,movies,t3_1coi02o,1.715319e+09,/r/movies/comments/1coi02o/bottoms_some_thoughts/,t2_5fw4514m,Discussion,Bottoms - some thoughts,It was a super fun movie and I enjoyed it a lo...,0,1
1,movies,t3_1cohlro,1.715317e+09,/r/movies/comments/1cohlro/presenting_the_chro...,t2_cq7rp7m1b,Discussion,Presenting the chronological viewing order of ...,This cinematic universe is arguably the greate...,5,1
2,movies,t3_1coh6ks,1.715316e+09,/r/movies/comments/1coh6ks/new_poster_for_a_qu...,t2_ruw91ssi8,Poster,New poster for ‘A QUIET PLACE: DAY ONE’,NaN,9,0
3,movies,t3_1coh5hv,1.715316e+09,/r/movies/comments/1coh5hv/what_is_your_favori...,t2_i94zymonh,Discussion,What is your favorite movie quote of all time.,Not sure if it's my favorite of all time but i...,58,17
4,movies,t3_1cogzod,1.715315e+09,/r/movies/comments/1cogzod/i_saw_godzilla_minu...,t2_kmbj6,Discussion,I saw Godzilla Minus One,I know people are going to hate this and proba...,17,0


In [24]:


# Get relevant data
data["title"] = df["title"].fillna("")
data["body"] = df["body"].fillna("")

# Combine title and body
data["text"] = data["title"] + " " + data["body"]

# Remove links
data["processed"] = data["text"].map(lambda x: re.sub(r"http\S+", "", x))

# Remove punctuation
# data["processed"] = data["processed"].map(lambda x: x.translate(str.maketrans("", "", string.punctuation)))

# Convert to lowercase
data["processed"] = data["processed"].map(lambda x: x.lower())

# Tokenize
data["processed"] = data["processed"].map(word_tokenize)

# Remove stopwords
# stop_words = set(stopwords.words("english"))
# data["processed"] = data["processed"].map(lambda x: [word for word in x if word not in stop_words])

# # Remove words with less than 3 characters
# data["processed"] = data["processed"].map(lambda x: [word for word in x if len(word) >= 3])

# Lemmatize
lemmatizer = WordNetLemmatizer()
data["processed"] = data["processed"].map(lambda x: [lemmatizer.lemmatize(word) for word in x])

# Stemming
# stemmer = PorterStemmer()
# data['processed'] = data['processed'].map(lambda x: [stemmer.stem(word) for word in x])

# Remove rows with less than 5 words
data = data[data["processed"].map(len) >= 10]

# Join words back to sentences
data["processed"] = data["processed"].map(lambda x: " ".join(x))

# Only keep text and processed columns
data = data[["text", "processed"]]

# reset index
data.reset_index(drop=True, inplace=True)


In [25]:
data.head()

,text,processed
0,Bottoms - some thoughts It was a super fun mov...,bottom - some thought it wa a super fun movie ...
1,Presenting the chronological viewing order of ...,presenting the chronological viewing order of ...
2,New poster for ‘A QUIET PLACE: DAY ONE’,new poster for ‘ a quiet place : day one ’
3,What is your favorite movie quote of all time....,what is your favorite movie quote of all time ...
4,I saw Godzilla Minus One I know people are goi...,i saw godzilla minus one i know people are goi...


In [27]:
from bertopic import BERTopic

topic_model = BERTopic(language="english", calculate_probabilities=True, verbose=True)
topics, probs = topic_model.fit_transform(data)

OSError: [WinError 126] The specified module could not be found. Error loading "C:\Users\BEA YUKDAWAN\AppData\Roaming\Python\Python311\site-packages\torch\lib\shm.dll" or one of its dependencies.